In [ ]:
# Hugging Face üzerinden model eğitmek ve paylaşmak için bir hesap oluşturmalısın:

# Hugging Face sitesine kaydol.
# Access Token oluştur (Profil > Settings > Access Tokens).
# Terminal veya Colab'da giriş yap:

In [ ]:
# 1.Gerekli Kütüphaneleri Kur
#pip install transformers datasets evaluate accelerate torch

In [ ]:
# 2. Hugging Face'e Giriş Yap
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 3. Load the dataset (IMDB Veri Setini Yükle)
from datasets import load_dataset

dataset = load_dataset("csv", data_files="dataset.csv")
print(dataset)

In [ ]:
# 4. Model ve Tokenizer Seç
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [4]:
# 5.Veriyi Tokenize Etme
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# 6.Eğitim Ayarlarını Yap
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],  # Küçük veri olduğu için test seti eklemedik
    data_collator=data_collator
)



In [6]:
# 7. Modeli Eğit
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.650385
2,No log,0.619884
3,No log,0.597047


TrainOutput(global_step=3, training_loss=0.6072676181793213, metrics={'train_runtime': 12.7573, 'train_samples_per_second': 1.176, 'train_steps_per_second': 0.235, 'total_flos': 3946665830400.0, 'train_loss': 0.6072676181793213, 'epoch': 3.0})

In [7]:
# 8. Modeli Hugging Face’e Yükle
trainer.push_to_hub("results")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tolgaege/results/commit/beae7f7304f127d0c112ad866322e66831e337e7', commit_message='results', commit_description='', oid='beae7f7304f127d0c112ad866322e66831e337e7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tolgaege/results', endpoint='https://huggingface.co', repo_type='model', repo_id='tolgaege/results'), pr_revision=None, pr_num=None)

In [ ]:
# 9. Modeli Test Et
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="tolgaege/results")
print(classifier("Bu film harikaydı!"))

In [ ]:
# Modeli yükleme
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi

# Hugging Face'e giriş yap (daha önce yapmadıysan)
!huggingface-cli login 

# Modeli yüklemeden önce Hugging Face deposunu oluştur
api = HfApi()
api.create_repo("tolgaege/results")

# Modeli yükle
model = AutoModelForSequenceClassification.from_pretrained("./results")  # Model dosyanın olduğu yol
tokenizer = AutoTokenizer.from_pretrained("./results")  

model.push_to_hub("tolgaege/results")
tokenizer.push_to_hub("tolgaege/results")

In [15]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Modelin ve tokenizer'ın bulunduğu yol
model_name = "bert-base-uncased"  # Modelin adı veya yolunun doğru olduğundan emin olun

# Modeli ve tokenizer'ı tekrar yükleyin
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [20]:
from transformers import pipeline

# Sentiment analysis pipeline'ını başlat
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Test mesajını analiz et
print(classifier("Bu film harikaydı!"))

Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.5247048735618591}]
